### <font color='#4285f4'>Overview</font>

**Overview**: Sensitive Data Protection (DLP) - Creates a sensitive data protection scan and identifies sensitive data.  Places the results in a new dataset which you can then easily use for reporting.

**Process Flow**:
1.  **Create a new dataset** to store the scan results.

2.  **Create a data profiling scan** for each of the 4 tables in the raw dataset:
    *   customer
    *   customer_transition
    *   product
    *   product_category

3.  **Wait for the scan to complete** (specifically, for the `customer` table scan).

4.  **Review the newly created BigQuery dataset** containing the scan results.

5.  **(Optional) Delete the scans.**

Notes:
* You can also schedule these scans. 

Cost:
* Approximate cost: Less than a dollar

Author:
* Adam Paternostro

In [ ]:
# Architecture Diagram
from IPython.display import Image
Image(url='https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/BigQuery-Data-Governance-SDP-Scan.png', width=1200)

### <font color='#4285f4'>Video Walkthrough</font>

[Video](https://storage.googleapis.com/data-analytics-golden-demo/colab-videos/Sensitive-Data-Protection-Scan.mp4)


In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/data-analytics-golden-demo/colab-videos/Sensitive-Data-Protection-Scan.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

### <font color='#4285f4'>License</font>

```
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

### <font color='#4285f4'>Initialize</font>

In [ ]:
from PIL import Image
from IPython.display import HTML
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random
import logging

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "${bigquery_location}"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### <font color='#4285f4'>Helper Methods</font>

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str, project_id=None) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import google.auth.transport.requests
  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  # Required by some API calls
  if project_id != None:
    headers["x-goog-user-project"] = project_id

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### RunQuery
Runs a SQL statement against BigQuery

In [ ]:
def RunQuery(sql):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      raise Exception(query_job.error_result)

### <font color='#4285f4'>Sensitive Data Protection - Helper Methods</font>

#### existsSDPScan
- Test to see if a scan exists
- NOTE: It prefixes the scan name with an "i-" ("i" stands for an inspection scan)
- If the scan exists, the does nothing

In [ ]:
def existsSDPScan(project_id, sdp_scan_name):
  """Test to see if a scan exists."""

  # Gather existing  scans
  # https://cloud.google.com/sensitive-data-protection/docs/reference/rest/v2/projects.dlpJobs/list

  url = f"https://dlp.googleapis.com/v2/projects/{project_id}/dlpJobs"

  # Gather existing scans
  json_result = restAPIHelper(url, "GET", None, project_id=project_id)
  print(f"existsSDPScan (GET) json_result: {json_result}")

  # Test to see if data scan exists, if so return
  if "jobs" in json_result:
    for item in json_result["jobs"]:
      print(f"Scan names: {item['name']}")
      if item["name"] == f"projects/{project_id}/dlpJobs/i-{sdp_scan_name}":
        print(f"SDP Scan {sdp_scan_name}  exists")
        return True

  return False

#### createSDPScan
- Creates a sensitive data proctection scan and starts to run it
- If the scan exists, the does nothing
- Returns the scan name

In [ ]:
def createSDPScan(project_id, sdp_scan_name, source_dataset, source_table, primary_key_name, destination_dataset, destination_table):
  """Creates a scan if it does not exist"""

  # publishFindingsToCloudDataCatalog

  if existsSDPScan(project_id,sdp_scan_name) == False:
    # https://cloud.google.com/sensitive-data-protection/docs/reference/rest/v2/projects.dlpJobs/create

    url = f"https://dlp.googleapis.com/v2/projects/{project_id}/dlpJobs"

    # Info Types: https://cloud.google.com/sensitive-data-protection/docs/infotypes-reference

    data = {
      "jobId": sdp_scan_name,

      "inspectJob":{
        "storageConfig":{
          "bigQueryOptions":{
            "tableReference":{
              "projectId": project_id,
              "datasetId": source_dataset,
              "tableId": source_table
            },
            "identifyingFields":[
              {
                "name": primary_key_name
              }
            ],
            "rowsLimit" : 0 # all rows
          }
        },

        "inspectConfig":{
          "infoTypes":[
            {
              "name":"CREDIT_CARD_NUMBER"
            },
            {
              "name":"DATE_OF_BIRTH"
            },
            {
              "name":"EMAIL_ADDRESS"
            },
            {
              "name":"FIRST_NAME"
            },
            {
              "name":"LAST_NAME"
            },
            {
              "name":"GENDER"
            },
            {
              "name":"IP_ADDRESS"
            },
            {
              "name":"DATE"
            }
          ],
          "excludeInfoTypes": False,
          "includeQuote": True,
          "minLikelihood": "LIKELY"
        },

        "actions":[
          {
            "saveFindings":{
              "outputConfig":{
                "table":{
                  "projectId": project_id,
                  "datasetId": destination_dataset,
                  "tableId": destination_table
                },
                "outputSchema": "BASIC_COLUMNS"
              }
            }
          },
          {
              "publishFindingsToCloudDataCatalog" : {}
          }
        ]
      }
    }

    result = restAPIHelper(url, "POST", request_body=data, project_id=project_id)
    print(result)
    sdp_job_name = result["name"]
  else:
    print(f"SDP Scan Already Exists")

#### getSDPScan
- Test to see if a scan exists
- If the scan exists, returns the scan data

In [ ]:
def getSDPScan(project_id, sdp_scan_name):
  """Gets a scan (in order to check its state)"""

  if existsSDPScan(project_id,sdp_scan_name) == True:
    # Get Scan
    # https://cloud.google.com/sensitive-data-protection/docs/reference/rest/v2/projects.dlpJobs/get

    url = f"https://dlp.googleapis.com/v2/projects/{project_id}/dlpJobs/i-{sdp_scan_name}"

    # Gather existing scans
    json_result = restAPIHelper(url, "GET", None, project_id=project_id)
    print(f"getSDPScan (GET) json_result: {json_result}")
    return json_result

#### deleteSDPScan
- Test to see if a scan exists
- If the scan exists, delete the scan

In [ ]:
def deleteSDPScan(project_id, sdp_scan_name):
  """Delete a scan"""

  if existsSDPScan(project_id,sdp_scan_name) == True:
    # Delete Scan
    # https://cloud.google.com/sensitive-data-protection/docs/reference/rest/v2/projects.dlpJobs/delete

    url = f"https://dlp.googleapis.com/v2/projects/{project_id}/dlpJobs/i-{sdp_scan_name}"

    # Gather existing scans
    json_result = restAPIHelper(url, "DELETE", None, project_id=project_id)
    print(f"deleteSDPScan (DELETE) json_result: {json_result}")

### <font color='#4285f4'>Run Sensitive Data Protection Scan (for a BigQuery table)</font>

- Creates a new dataset to store the results
- Creates a new scan for the customer table
- Starts the scan (after a delay)
- Monitors the scans progress

In [ ]:
# Create a new dataset to hold the SDP results (keep seperate from source tables)

governed_data_sdp_scan_dataset_name = "governed_data_sdp_scan"

sql = f"""CREATE SCHEMA IF NOT EXISTS `{project_id}.{governed_data_sdp_scan_dataset_name}` OPTIONS(location="{bigquery_location}")"""

RunQuery(sql)

In [ ]:
# Create the SDP scan on the customer table

customer_sdp_scan_name = "sdp-governed-data-raw-customer"
source_dataset = "${bigquery_governed_data_raw_dataset}"
source_table = "customer"
primary_key_name = "customer_id"
destination_dataset = governed_data_sdp_scan_dataset_name
destination_table = "customer"

createSDPScan(project_id, customer_sdp_scan_name, source_dataset, source_table, primary_key_name, destination_dataset, destination_table)

In [ ]:
# Show the Customer scan

getSDPScan(project_id, customer_sdp_scan_name)

In [ ]:
# Create the SDP scan on the customer_transaction table

customer_transaction_sdp_scan_name = "sdp-governed-data-raw-customer-transaction"
source_dataset = "${bigquery_governed_data_raw_dataset}"
source_table = "customer_transaction"
primary_key_name = "transaction_id"
destination_dataset = governed_data_sdp_scan_dataset_name
destination_table = "customer_transaction"

createSDPScan(project_id, customer_transaction_sdp_scan_name, source_dataset, source_table, primary_key_name, destination_dataset, destination_table)

In [ ]:
# Create the SDP scan on the product table

product_sdp_scan_name = "sdp-governed-data-raw-project"
source_dataset = "${bigquery_governed_data_raw_dataset}"
source_table = "product"
primary_key_name = "product"
destination_dataset = governed_data_sdp_scan_dataset_name
destination_table = "product"

createSDPScan(project_id, product_sdp_scan_name, source_dataset, source_table, primary_key_name, destination_dataset, destination_table)

In [ ]:
# Create the SDP scan on the product_category table

product_category_sdp_scan_name = "sdp-governed-data-raw-product-category"
source_dataset = "${bigquery_governed_data_raw_dataset}"
source_table = "product_category"
primary_key_name = "product_category"
destination_dataset = governed_data_sdp_scan_dataset_name
destination_table = "product_category"

createSDPScan(project_id, product_category_sdp_scan_name, source_dataset, source_table, primary_key_name, destination_dataset, destination_table)

In [ ]:
# Monitor the Customer scan (it automatically starts so we monitor its progress)
print(f"You can see your scan jobs here: https://console.cloud.google.com/security/sensitive-data-protection/landing/inspection/jobs?hl=en&inv=1&invt=Abn-xA&project={project_id}&supportedpurview=project")
print()

sdp_scan_state = getSDPScan(project_id, customer_sdp_scan_name)
print(f"sdp_scan_state: {sdp_scan_state}")

while sdp_scan_state["state"] == "PENDING" or \
      sdp_scan_state["state"] == "JOB_STATE_UNSPECIFIED" or \
      sdp_scan_state["state"] == "RUNNING" or \
      sdp_scan_state["state"] == "ACTIVE" or \
      sdp_scan_state["state"] == "CANCELING":
  time.sleep(10)
  sdp_scan_state = getSDPScan(project_id, customer_sdp_scan_name)
  print(f"sdp_scan_state: {sdp_scan_state}")

print()
print("Customer Table: SDP Scan complete.  You should see a new BigQuery dataset.")
print("The other tables might still be in progress.  The product and product_category do not have sensitive data and will be empty.")
print()
print("")
print(f"You can see your tag templates here: https://console.cloud.google.com/dataplex/templates?hl=en&inv=1&invt=Abn-xA&project={project_id}&supportedpurview=project")

### <font color='#4285f4'>Clean Up</font>

In [ ]:
# Delete the scan

user_input = input(f"Do you want to delete your SDP scans(Y/n)?")
if user_input == "Y":
  print("This will not delete the dataset created by the scan.")
  deleteSDPScan(project_id,customer_sdp_scan_name)
  deleteSDPScan(project_id,customer_transaction_sdp_scan_name)
  deleteSDPScan(project_id,product_sdp_scan_name)
  deleteSDPScan(project_id,product_category_sdp_scan_name)

### <font color='#4285f4'>Reference Links</font>


- [REPLACE-ME](https://REPLACE-ME)